In [1]:
import pandas as pd
import json
import csv
from tqdm import tnrange, tqdm_notebook

In [2]:
f = open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/for_coding2.json', 'r', encoding='UTF8')    # type3를 통해 추출된 json파일 읽어들이기
original_exp = json.load(f)
key_list = sorted(list(original_exp.keys()), key=len)    # 어려운 단어 리스트. 글자수가 더 많은 키워드를 먼저 ^^로 태깅하면 문제가 생김. 그래서 오름차순 정렬함

In [3]:
# 단어 하나가 문장에 여러번 등장했을때, 모든 위치를 리스트로 뽑아내는 함수.
# list씌워야함

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches
        
# ex) list(find_all('안녕안녕하세요', '안녕')) --> [0,2]

In [4]:
# isdiff 두 문장을 갖고와서 같으면 0, 다르면 1을 출력하는 함수.
def isdiff(sent_a, sent_b):
    if sent_a == sent_b:
        return 0
    else:
        return 1

In [5]:
# 숫자 리스트 받아와서 이에 맞게 다른 리스트의 순서 바꿔주는 함수.
def sort_word_wordOcc(word, wordOccurence):    # 순서 바꿀 단어 리스트, 순서 알려주는 리스트
#     try:
    if len(word) == len(wordOccurence):
        result = {}
        for i in range(len(word)):
            result.update({wordOccurence[i]['start']:word[i]})    # result = {wordOccurence:['start']:호환}

        sorted_result = sorted(result.items())     # ex) [('1','원심'), (17:'원심'), (127:'호환'), (172:'호환'), (320:'원심')]
        result_word = []
        result_wordOcc = []

        for i in range(0, len(word)):
            result_word.append(sorted_result[i][1])
            result_wordOcc.append({'start':sorted_result[i][0], 'end':sorted_result[i][0]+len(sorted_result[i][1])+1})
        
    else:
        raise Exception('word와 word_Occurence 간 크기가 다릅니다.')
        
#     except Exception as e:
#         print("예외가 발생했습니다.", e)
        
    return result_word, result_wordOcc

In [6]:
# 중복제거
def remove_overlap(overlapped_list):
    result = []
    for i in overlapped_list:
        if i not in result:
            result.append(i)
            
    return result

In [7]:
result = {}
tag = '^'

for i in tnrange(len(key_list)):
    result.update({key_list[i]:[]})
    
    for j in range(len(original_exp[key_list[i]])):
        sentence = original_exp[key_list[i]][j]["sentence"]
        paragraph = original_exp[key_list[i]][j]["paragraph"]
        keyword = original_exp[key_list[i]][j]["keyword"]
        keywordOccurence = []
        
        for k in range(len(paragraph)):
            sentence = sentence.replace(paragraph[k], tag + keyword[k] + tag)
            
        for q in range(len(paragraph)):
            start = list(find_all(sentence, keyword[q]))
            
            for l in range(len(start)):
                keywordOccurence.append({'start':start[l]-1, 'end':start[l]+len(keyword[k])})
            
        result[key_list[i]].append({'sentence':sentence, 'paragraph': paragraph, 'keyword':keyword, 'keywordOccurence':keywordOccurence})
        
#         # 키데이터 중 문장에 있는 키워드 찾아내기
#         for k in range(len(key_list)):    # 모든 키워드에 대해 시행
#             if sentence[0:len(key_list[k])] == key_list[k]:    # 문장 첫 단어로 키워드가 있을 때..
#                 tagged_keyword = tag + data1[3] + tag
#                 sentence = sentence.replace(key_list[k], tagged_keyword)
#                 count = len(list(find_all(sentence, key_list[k])))
#                 result[key_list[i]][j].update({'sentence':sentence})
#                 for l in range(count):
#                     result[key_list[i]][j]['keyword'].append(key_list[k])
                
                
#             elif sentence.find(' '+key_list[k]) != -1:    # 문장 첫 단어로 키워드가 있을 때..
#                 tagged_keyword = tag + key_list[k] + tag
#                 sentence = sentence.replace(key_list[k], tagged_keyword)
#                 count = len(list(find_all(sentence, key_list[k])))
#                 result[key_list[i]][j].update({'sentence':sentence})
#                 for l in range(count):
#                     result[key_list[i]][j]['keyword'].append(key_list[k])
                
#             else:
#                 pass

<ipython-input-7-8c697acb1443>:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(key_list)):


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# word와 wordOccurence 직관적으로 정렬.
for i in tnrange(len(key_list)):
    for j in range(len(original_exp[key_list[i]])):
        paragraph = result[key_list[i]][j]['paragraph']
        keyword = result[key_list[i]][j]['keyword']
        tagged_keyword_pos = result[key_list[i]][j]['keywordOccurence']
        result_word, result_wordOcc = sort_word_wordOcc(paragraph, tagged_keyword_pos)
        result[key_list[i]][j]['paragraph'] = result_word
        result_word, result_wordOcc = sort_word_wordOcc(keyword, tagged_keyword_pos)
        result[key_list[i]][j]['keyword'] = result_word
        result[key_list[i]][j]['keywordOccurence'] = result_wordOcc

<ipython-input-8-cb0679df4dae>:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(key_list)):


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# json 파일로 저장
with open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/type2_B_input.json', 'w', encoding='UTF8') as json_file:
# with open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/type2_A_input_example.json', 'w', encoding='UTF8') as json_file:
    json.dump(result, json_file, indent=4, ensure_ascii=False,)

In [10]:
result_sentence = {}
for i in tnrange(len(key_list)):
    result_sentence.update({key_list[i]:[]})
    for j in range(len(original_exp[key_list[i]])):
        sentence = result[key_list[i]][j]["sentence"]
        result_sentence[key_list[i]].append({'sentence':sentence})

<ipython-input-10-b5c48387b1fa>:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(key_list)):


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# 앱 업로드용 csv 파일로 저장
with open('C:/Users/jayts/OneDrive/바탕 화면/Tictag/json/KAS/type2_B_app_input.csv', 'w', newline='') as file:
    fieldnames=['sentence']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(result_sentence)):
        for j in range(len(result_sentence[key_list[i]])):
            writer.writerow(result_sentence[key_list[i]][j])

In [12]:
result_sentence

{'기금': [{'sentence': '[1] 증권투자신탁업법 제59조 제1호、 제4조는 "그 법에 의하지 아니하고 현금 등의 재산을 납입받아 유가증권에 투자 운용하고 그 수익권을 분할하여 불특정 다수인에게 취득시키는 업무를 행하는 것"을 처벌하고 있는바、 이에 해당하기 위하여는 우선 투자자들로부터 납입받은 재산을 유가증권에 투자하는 것이어야 하고、 또 투자자들로부터 납입받은 재산으로 일정한 ^기초 자금을 만드는 것하고^ 그 기금으로부터 발생하는 수익 또는 손실이 각 참여 투자자들에게 투자금액에 비례하여 귀속하는 방식으로 운용되는 것이어야 한다.'},
  {'sentence': '[2] ^손님이 증권을 다루는 회사에^ 대한 채권이 증권투자자보호기금의 지급대상임을 주장하면서 증권회사에 권리신고 및 지급청구를 하였으나 증권회사에서 그 채권이 증권투자자보호기금의 지급대상이 아니라는 이유로 그 신고접수를 거절할 경우 고객으로서는 증권관리위원회에 자신의 채권이 그 지급대상에 해당함의 확인을 구하고、 증권관리위원회의 결정에 따라 그 채권을 ^기초자금운용회사로부터 지급받아야^ 하고、 만약 증권관리위원회가 그 채권이 증권투자자보호기금의 지급대상이 아니라고 판단할 경우 구 증권거래법(1998. 1. 8. 법률 제5498호로 개정되기 전의 것) 제206조에 정하여진 절차에 따라 재정경제부장관을 대상으로 행정심판을 청구하는 등의 불복절차를 밟아야 한다.'}],
 '각서': [{'sentence': '[2] 증권회사 직원이 증권투자로 인한 고객의 손해에 대하여 책임을 지겠다는 내용의 ^쌍방의 합의가 약속된 문서를 작성해^ 준 사안에서、 그 각서를 단지 그 동안의 손실에 대하여 사과하고 그 회복을 위해 최선을 다하겠다는 의미로 해석하는 것은 경험칙과 논리칙에 반하지만、 그 각서가 남편을 안심시키려는 고객의 요청에 따라 작성된 경위 등에 비추어 비진의 의사표시로서 무효라고 본 사례'},
  {'sentence': '[1] 수사 도중에 피의자가 수사관에게 소유권포기 ^쌍방의 합의가 약속